In [15]:
import os
import json
import pandas as pd

folder = "./sg-paramtuning"

def calculate_incr(history, key):
    values = [el[key] for el in history]
    deltas = [values[i+1] - values[i] for i in range(len(values)-1)]
    return sum(deltas) / len(deltas)

df = []

for fname in os.listdir(folder):
    path = os.path.join(folder, fname)
    data = json.load(open(path))
    entry = {
        "name" : fname,
        "test_f1" : data["evaluation_metrics"]["test_sg_f1"],
        "train_f1_incr" : calculate_incr(data["history"], "train_sg_f1"),
        "val_f1_incr" : calculate_incr(data["history"], "val_sg_f1"),
        "train_f1_last" : data["history"][-1]["train_sg_f1"],
        "val_f1_last" : data["history"][-1]["val_sg_f1"]
    }
    df.append(entry)

df = pd.DataFrame(df)

In [16]:
df

,name,test_f1,train_f1_incr,val_f1_incr,train_f1_last,val_f1_last
0,0.json,0.491393,0.001142,-0.000487,0.585103,0.485425
1,1.json,0.485861,-0.001651,-0.000806,0.499163,0.486617
2,10.json,0.618705,0.057978,0.019967,0.750148,0.640458
3,11.json,0.506023,-0.002147,-0.008893,0.546512,0.506491
4,12.json,0.491386,0.002028,-0.000330,0.585744,0.486855
...,...,...,...,...,...,...
67,7.json,0.490236,-0.001595,-0.001634,0.504553,0.487694
68,70.json,0.630571,0.057199,0.023700,0.734954,0.644131
69,71.json,0.541695,0.001969,0.014465,0.559803,0.528145
70,8.json,0.503050,0.030076,0.006228,0.608978,0.513131


In [26]:
def get_best(df, column):
    print(df.loc[df[column].argmax(), "name"])

    with open(os.path.join(folder,df.loc[df[column].argmax(),"name"]), 'r') as fp:
        best_test_f1 = json.load(fp)
    print(best_test_f1["hparam"])

    print(df.loc[df[column].argmax()])

In [29]:
for c in df.columns:
    if c == "name":
        continue
    print(f">>>{c}")
    get_best(df, c)
    print("="*32)

>>>test_f1
70.json
{'model_name_or_path': 'meta-llama/Meta-Llama-3-8B', 'h_dim': 128, 'n_injector_head': 4, 'injector_dropout_p': 0.1, 'encoder_dropout_p': 0.1, 'n_encoder_head': 4, 'n_encoder_layers': 2, 'to_matrix': 'diagonal'}
name              70.json
test_f1          0.630571
train_f1_incr    0.057199
val_f1_incr        0.0237
train_f1_last    0.734954
val_f1_last      0.644131
Name: 68, dtype: object
>>>train_f1_incr
22.json
{'model_name_or_path': 'meta-llama/Meta-Llama-3-8B', 'h_dim': 512, 'n_injector_head': 4, 'injector_dropout_p': 0.1, 'encoder_dropout_p': 0.1, 'n_encoder_head': 4, 'n_encoder_layers': 2, 'to_matrix': 'diagonal'}
name              22.json
test_f1          0.577909
train_f1_incr    0.060832
val_f1_incr     -0.001235
train_f1_last    0.763222
val_f1_last      0.601174
Name: 15, dtype: object
>>>val_f1_incr
58.json
{'model_name_or_path': 'meta-llama/Meta-Llama-3-8B', 'h_dim': 128, 'n_injector_head': 8, 'injector_dropout_p': 0.1, 'encoder_dropout_p': 0.1, 'n_encode